In [1]:
# 주요 라이브러리 
import copy
import re
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [24]:
# 커널 로그를 기반으로 패킷 송수신 내용 추출
def read(file_name):
    # 파일 읽기
    with open(file_name) as json_file:
        message = json_file.read()
        
    # 정규식 추출
    # 예시: tcp_header_size,-544,507,48
    # 데이터 이름, Flow id, 시간, 값
    p = re.compile('''([a-z\_0-9]+),([\-0-9]+),([0-9]+),([0-9]+)''')
    result = p.finditer(message)
    
    temp = []
    for r in result:
        key = str(r.group(1))
        flow_id = int(r.group(2))
        flow_time = int(r.group(3))
        value = int(r.group(4))
        temp.append([flow_time, flow_id, key, value])

    # [time, flow_id, key, value] 형태의 테이블 생성
    # 그러나 MPTCP가 아닌 다른 플로우 데이터도 함께 존재함
    table = pd.DataFrame(temp, columns=['time', 'flow_id', 'key', 'value'])
    
    # 가장 많은 데이터를 보낸 3개의 플로우를 제외한 나머지 제거
    table_not_included_meta_flow = table[table['flow_id'] != 0]
    remove_id = table_not_included_meta_flow['flow_id'].value_counts().keys()[2:]
    table = table.loc[table['flow_id'].isin(remove_id) == False]
    
    # 첫 번째 subflow 찾기 (=가장 처음으로 로그가 발생한 플로우)
    temp = table[table['flow_id'] != 0] # Meta 레벨이 아닌 것 중
    subflow_id = temp.sort_values(["time"], ascending=[True])
    first_flow = subflow_id.iloc[0]['flow_id']
    
    # 각 flow별로 네임태그 지정
    table.loc[table['flow_id'] == 0, 'flow_id'] = 'mptcp'
    table.loc[table['flow_id'] == first_flow, 'flow_id'] = 'sub1'
    table.loc[(table['flow_id'] != 'mptcp') & (table['flow_id'] != 'sub1') , 'flow_id'] = 'sub2'
    
    # 시간 값 정규화 (커널 시간 -> FLow 시작 시간)
    min_time = table.loc[:,"time"].min() # 기준 값 설정
    
    # 250hz의 세분성
    table.loc[:,"time"] = (table.loc[:,"time"] - min_time) / 250
    
    table.name = file_name
    return table

In [25]:
prefix = "logs/"
default = read(prefix + "redundant00165.txt");

In [23]:
default

,time,flow_id,key,value
15,512,mptcp,rtt,20
16,512,mptcp,srtt,20
33,537,sub1,send,130
34,537,sub1,tcp_header_size,48
35,537,sub1,cwnd,10
...,...,...,...,...
1695471,8093,sub1,ssthresh,2147483647
1695472,8093,mptcp,receive_seq,368641145
1695481,8096,sub1,receive,0
1695482,8096,sub1,rtt,200
